<h1> Amazon Apparel Recommendations Engine</h1>

Skills: Python, text processing, image processing

# Problem Statement

Recommendation engine which suggests  similar products to the given product in any e-commerce websites ex. Amazon.com, myntra.com etc.

Ex.Recommend similar apparel products using product descriptions and Images

<tr> 
<td><img src="apperalrecommend.png",width=300,height=300> </td>
</tr>

In this project I will build a recommendation engine that suggests relevant apparels to the given apparel.The recommendation engine, uses information about 1,80,000 products and  each product will have multiple features named.

1.Title of the product <br> 
2.Brand of the product <br>
3.Color of the product <br>
4.Type of the product <br>
5.Image of the apparel <br>

Data Source: Amazon.com



### Exploring the data(Data Understanding)
Let’s start with importing the necessary libaries, reading the data, and analyze the dataset.

In [1]:
#import all the necessary packages.

import numpy as np
import pandas as pd
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [2]:

#Load the amazon tops_fashion json file which consists of all information about the products
data = pd.read_json('tops_fashion.json')

# Success - Display the first record
display(data.head(n=1))

,asin,author,availability,availability_type,brand,color,editorial_reivew,editorial_review,formatted_price,large_image_url,manufacturer,medium_image_url,model,product_type_name,publisher,reviews,sku,small_image_url,title
0,B016I2TS4W,None,None,None,FNC7C,None,NaN,Minions Como Superheroes Ironman Women's O Nec...,None,https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,None,SHIRT,None,"[False, https://www.amazon.com/reviews/iframe?...",None,https://images-na.ssl-images-amazon.com/images...,Minions Como Superheroes Ironman Long Sleeve R...


In [3]:
print ('Number of data points : ', data.shape[0], \
       '\nNumber of features/variables:', data.shape[1],'\n\n')

#Checking out the datatypes of the features
data.info()

Number of data points :  183138 
Number of features/variables: 19 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 183138 entries, 0 to 183137
Data columns (total 19 columns):
asin                 183138 non-null object
author               1 non-null object
availability         24532 non-null object
availability_type    24559 non-null object
brand                182987 non-null object
color                64956 non-null object
editorial_reivew     180380 non-null object
editorial_review     2758 non-null object
formatted_price      28395 non-null object
large_image_url      183138 non-null object
manufacturer         42899 non-null object
medium_image_url     183138 non-null object
model                62370 non-null object
product_type_name    183138 non-null object
publisher            42899 non-null object
reviews              183138 non-null object
sku                  363 non-null object
small_image_url      183138 non-null object
title                183138 non-null object
d

#### A simple investigation of the dataset can determine many irrlevent features in dataset such as author, availability .

Of these 19 features, I am using only 7 features.
    1. asin  ( Amazon standard identification number)
    2. brand ( brand to which the product belongs to )
    3. color ( Color information of apparel, it can contain many colors as   a value ex: red and black stripes ) 
    4. medium_image_url  ( url of the image )
    5. product_type_name (type of the apperal, ex: SHIRT/TSHIRT )
    6. title (title of the product.)
    7. formatted_price (price of the product)
    
So,Let's shrink our dataset to the only following features.

In [4]:
data = data[['asin', 'brand', 'color', 'medium_image_url', 'product_type_name', 'title', 'formatted_price']]

After shrinking no of features in dataset our dataset have following fetures. Heare I am printing top 3 records of dataset.

In [5]:
print ('Number of data points : ', data.shape[0], \
       '\nNumber of features:', data.shape[1])
data.head(3) # prints the top 3 rows in the table.

Number of data points :  183138 
Number of features: 7


,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
0,B016I2TS4W,FNC7C,None,https://images-na.ssl-images-amazon.com/images...,SHIRT,Minions Como Superheroes Ironman Long Sleeve R...,None
1,B01N49AI08,FIG Clothing,None,https://images-na.ssl-images-amazon.com/images...,SHIRT,FIG Clothing Womens Izo Tunic,None
2,B01JDPCOHO,FIG Clothing,None,https://images-na.ssl-images-amazon.com/images...,SHIRT,FIG Clothing Womens Won Top,None


## Analyzing various features of the dataset.

####  Basic stats for the feature: product_type_name

In [6]:
# We have total 72 unique type of product_type_names
print(data['product_type_name'].describe())

# 91.62% (167794/183138) of the products are shirts,


count     183138
unique        72
top        SHIRT
freq      167794
Name: product_type_name, dtype: object


So, from above stats we can see that each product have their product name since count is equal to no of records in our dataset. So,there is no any missing value for this feature and there is 72 unique product name in our dataset with most frequent is shirt.<br><br>
Here is name of the different product.

In [7]:
# names of different product types
print(data['product_type_name'].unique())

['SHIRT' 'SWEATER' 'APPAREL' 'OUTDOOR_RECREATION_PRODUCT'
 'BOOKS_1973_AND_LATER' 'PANTS' 'HAT' 'SPORTING_GOODS' 'DRESS' 'UNDERWEAR'
 'SKIRT' 'OUTERWEAR' 'BRA' 'ACCESSORY' 'ART_SUPPLIES' 'SLEEPWEAR'
 'ORCA_SHIRT' 'HANDBAG' 'PET_SUPPLIES' 'SHOES' 'KITCHEN' 'ADULT_COSTUME'
 'HOME_BED_AND_BATH' 'MISC_OTHER' 'BLAZER' 'HEALTH_PERSONAL_CARE'
 'TOYS_AND_GAMES' 'SWIMWEAR' 'CONSUMER_ELECTRONICS' 'SHORTS' 'HOME'
 'AUTO_PART' 'OFFICE_PRODUCTS' 'ETHNIC_WEAR' 'BEAUTY'
 'INSTRUMENT_PARTS_AND_ACCESSORIES' 'POWERSPORTS_PROTECTIVE_GEAR' 'SHIRTS'
 'ABIS_APPAREL' 'AUTO_ACCESSORY' 'NONAPPARELMISC' 'TOOLS' 'BABY_PRODUCT'
 'SOCKSHOSIERY' 'POWERSPORTS_RIDING_SHIRT' 'EYEWEAR' 'SUIT'
 'OUTDOOR_LIVING' 'POWERSPORTS_RIDING_JACKET' 'HARDWARE' 'SAFETY_SUPPLY'
 'ABIS_DVD' 'VIDEO_DVD' 'GOLF_CLUB' 'MUSIC_POPULAR_VINYL'
 'HOME_FURNITURE_AND_DECOR' 'TABLET_COMPUTER' 'GUILD_ACCESSORIES'
 'ABIS_SPORTS' 'ART_AND_CRAFT_SUPPLY' 'BAG' 'MECHANICAL_COMPONENTS'
 'SOUND_AND_RECORDING_EQUIPMENT' 'COMPUTER_COMPONENT' 'JEWELRY'
 'B

In [8]:
# find the 10 most frequent product_type_names.
product_type_count = Counter(list(data['product_type_name']))
product_type_count.most_common(10)

[('SHIRT', 167794),
 ('APPAREL', 3549),
 ('BOOKS_1973_AND_LATER', 3336),
 ('DRESS', 1584),
 ('SPORTING_GOODS', 1281),
 ('SWEATER', 837),
 ('OUTERWEAR', 796),
 ('OUTDOOR_RECREATION_PRODUCT', 729),
 ('ACCESSORY', 636),
 ('UNDERWEAR', 425)]

####  Basic stats for the feature: brand


In [9]:
print(data['brand'].describe())

count     182987
unique     10577
top         Zago
freq         223
Name: brand, dtype: object


Here in brand features there is missing valuse for some products(record) since count is only 182987 while total no of records is 183138.<br><br>
So, No of missing values = 183138 - 182987 = 151.<br>
And there is 10577 unique brand in dataset with Most frequent one is Zago.

In [10]:
#find the 10 most frequent brand
brand_count = Counter(list(data['brand']))
brand_count.most_common(10)

[('Zago', 223),
 ('XQS', 222),
 ('Yayun', 215),
 ('YUNY', 198),
 ('XiaoTianXin-women clothes', 193),
 ('Generic', 192),
 ('Boohoo', 190),
 ('Alion', 188),
 ('Abetteric', 187),
 ('TheMogan', 187)]

####  Basic stats for the feature: color

In [11]:
print(data['color'].describe())

count     64956
unique     7380
top       Black
freq      13207
Name: color, dtype: object


Only 64956 products have their color information.<br>
So, No of missing values = 183138-64956 = 118182.<br>
Only 35.4 % products have their color information.

In [12]:
#find the 10 most frequent color
color_count = Counter(list(data['color']))
color_count.most_common(10)

[(None, 118182),
 ('Black', 13207),
 ('White', 8616),
 ('Blue', 3570),
 ('Red', 2289),
 ('Pink', 1842),
 ('Grey', 1499),
 ('*', 1388),
 ('Green', 1258),
 ('Multi', 1203)]

####  Basic stats for the feature: formatted_price

In [13]:
 print(data['formatted_price'].describe())

count      28395
unique      3135
top       $19.99
freq         945
Name: formatted_price, dtype: object


Only 28,395 (15.5% of whole data) products have price information so there is lot of missing values for price feature in dataset.

In [14]:
#find the 10 most frequent price from dataset.
price_count = Counter(list(data['formatted_price']))
price_count.most_common(10)

[(None, 154743),
 ('$19.99', 945),
 ('$9.99', 749),
 ('$9.50', 601),
 ('$14.99', 472),
 ('$7.50', 463),
 ('$24.99', 414),
 ('$29.99', 370),
 ('$8.99', 343),
 ('$9.01', 336)]

#### Basic stats for the feature: title


In [15]:
print(data['title'].describe())

count                                                183138
unique                                               175985
top       Nakoda Cotton Self Print Straight Kurti For Women
freq                                                     77
Name: title, dtype: object


All of the products have a title feature in the dataset we can see with above outputs.<br>
<br>
Python have pickle files facilites in which we can save our data at any point of time. Since we have reduced the feature in dataset so we can save our dataset in pickle folder such that if we stuck at any point we can take directly data from pickle folder so that we don't need to repeat the all step again.

In [16]:
data.to_pickle('pickels/180k_apparel_data')

## Data cleaning and preprocessing

We have seen that in our dataset some of the features have missing values so we need to preprocess the dataset such that there should not be any inconsistency in our dataset and dataset can work efficiently with algorithms.<br>

### Missing value treatment.
    There is many methods for treatment of missing values but I am going with Deletion method in which we delete all the records where any of the variable is missing, if processing power of computer is high than we can  use another methods such as mean/median/mode imputation for missing value instead of deletion method.
   We have seen that feature price in our dataset have many missing values so we can remove all such records from our dataset which don't have price information.

In [17]:
# read data from pickle file from previous stage
data = pd.read_pickle('pickels/180k_apparel_data')

# data['formatted_price'].isnull() => gives the information 
#about the dataframe row's which have null values price == None|Null
data = data.loc[~data['formatted_price'].isnull()]
print('Number of data points After eliminating price=NULL :', data.shape[0])

Number of data points After eliminating price=NULL : 28395


 So our dataset have reduced to only 28K records after removing rcords which don't have price information.
Similarly we can remove all record which don't have color information since color feature have also missing values.

In [18]:
# data['color'].isnull() => gives the information about the dataframe row's which have null values price == None|Null
data =data.loc[~data['color'].isnull()]
print('Number of data points After eliminating color=NULL :', data.shape[0])

Number of data points After eliminating color=NULL : 28385


     I brought down the number of data points from 183K to 28K.
I am processing only 28K points so that my computer can run this code in a reasonable amount of time. <br>
Now I have less no of records in our dataset so I can pickle this processed dataset such that if there is any problems at any point of time I can directly take this dataset for further processing.

In [19]:
data.to_pickle('pickels/28k_apparel_data')

### Remove duplicate items
Since in our dataset many features/attribtes such as titles of the products will be same since many products will have multiple sizes for thier products.<br>
<br>
So, let's see duplicates products for feature titles in our dataset

In [20]:
# read data from pickle file from previous stage
data = pd.read_pickle('pickels/28k_apparel_data')

# find number of products that have duplicate titles.
print('Number of products that have duplicate titles =',sum(data.duplicated('title')))

Number of products that have duplicate titles = 2325


 So,we have 2325 products which have same title but These produts are exactly same except in size (S, M,L,XL). we need to de-dupe them for better results since our recommendation is very much depends on the titles of the products.

In [21]:
# Remove All products with very few words in title
data_sorted = data[data['title'].apply(lambda x: len(x.split())>4)]
print("After removal of products with short description:", data_sorted.shape[0])
data.head()

After removal of products with short description: 27949


,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
4,B004GSI2OS,FeatherLite,Onyx Black/ Stone,https://images-na.ssl-images-amazon.com/images...,SHIRT,Featherlite Ladies' Long Sleeve Stain Resistan...,$26.26
6,B012YX2ZPI,HX-Kingdom Fashion T-shirts,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,Women's Unique 100% Cotton T - Special Olympic...,$9.99
11,B001LOUGE4,Fitness Etc.,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,Ladies Cotton Tank 2x1 Ribbed Tank Top,$11.99
15,B003BSRPB0,FeatherLite,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,FeatherLite Ladies' Moisture Free Mesh Sport S...,$20.54
21,B014ICEDNA,FNC7C,Purple,https://images-na.ssl-images-amazon.com/images...,SHIRT,Supernatural Chibis Sam Dean And Castiel Short...,$7.50


Sort the whole dataset based on title (alphabetical order of title) such that it is easy to understand the dataset and also easy for deduping the similar products from dataset.


In [22]:

data_sorted.sort_values('title',inplace=True, ascending=False)
data_sorted.head()

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
61973,B06Y1KZ2WB,Éclair,Black/Pink,https://images-na.ssl-images-amazon.com/images...,SHIRT,Éclair Women's Printed Thin Strap Blouse Black...,$24.99
133820,B010RV33VE,xiaoming,Pink,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Womens Sleeveless Loose Long T-shirts...,$18.19
81461,B01DDSDLNS,xiaoming,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Women's White Long Sleeve Single Brea...,$21.58
75995,B00X5LYO9Y,xiaoming,Red Anchors,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Stripes Tank Patch/Bear Sleeve Anchor...,$15.91
151570,B00WPJG35K,xiaoming,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Sleeve Sheer Loose Tassel Kimono Woma...,$14.32


#### Some examples of dupliacte titles that differ only in the last few words.

<pre>
Titles 1:
16. woman's place is in the house and the senate shirts for Womens XXL White
17. woman's place is in the house and the senate shirts for Womens M Grey

Title 2:
25. tokidoki The Queen of Diamonds Women's Shirt X-Large
26. tokidoki The Queen of Diamonds Women's Shirt Small
27. tokidoki The Queen of Diamonds Women's Shirt Large

Title 3:
61. psychedelic colorful Howling Galaxy Wolf T-shirt/Colorful Rainbow Animal Print Head Shirt for woman Neon Wolf t-shirt
62. psychedelic colorful Howling Galaxy Wolf T-shirt/Colorful Rainbow Animal Print Head Shirt for woman Neon Wolf t-shirt
63. psychedelic colorful Howling Galaxy Wolf T-shirt/Colorful Rainbow Animal Print Head Shirt for woman Neon Wolf t-shirt
64. psychedelic colorful Howling Galaxy Wolf T-shirt/Colorful Rainbow Animal Print Head Shirt for woman Neon Wolf t-shirt
</pre>

So, Remove all these kinds of titles if the title is almost similar. I am assuming the same titles if it is only differ by 2 words in the titles of the products.

In [23]:
indices = []
for i,row in data_sorted.iterrows():
    indices.append(i)

In [24]:
import itertools
stage1_dedupe_asins = []
i = 0
j = 0
num_data_points = data_sorted.shape[0]
while i < num_data_points and j < num_data_points:
    
    previous_i = i

    # store the list of words of ith string in a, ex: a = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']
    a = data['title'].loc[indices[i]].split()

    # search for the similar products sequentially 
    j = i+1
    while j < num_data_points:

        # store the list of words of jth string in b, ex: b = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'Small']
        b = data['title'].loc[indices[j]].split()

        # store the maximum length of two strings
        length = max(len(a), len(b))

        # count is used to store the number of words that are matched in both strings
        count  = 0

        # itertools.zip_longest(a,b): will map the corresponding words in both strings, it will appened None in case of unequal strings
        # example: a =['a', 'b', 'c', 'd']
        # b = ['a', 'b', 'd']
        # itertools.zip_longest(a,b): will give [('a','a'), ('b','b'), ('c','d'), ('d', None)]
        for k in itertools.zip_longest(a,b): 
            if (k[0] == k[1]):
                count += 1

        # if the number of words in which both strings differ are > 2 , I am considering it as those two apperals are different
        # if the number of words in which both strings differ are < 2 ,  I am considering it as those two apperals are same, hence I am ignoring them
        if (length - count) > 2: # number of words in which both sensences differ
            # if both strings are differ by more than 2 words I am including the 1st string index
            stage1_dedupe_asins.append(data_sorted['asin'].loc[indices[i]])

            # if the comaprision between num_data_points and num_data_points-1 strings and they differ in more than 2 words than include both
            if j == num_data_points-1: stage1_dedupe_asins.append(data_sorted['asin'].loc[indices[j]])

            # start searching for similar apperals corresponds 2nd string
            i = j
            break
        else:
            j += 1
    if previous_i == i:
        break

Now reduce the dataset with only those products which have the same asins in the deduped dataset

In [25]:
data = data.loc[data['asin'].isin(stage1_dedupe_asins)]
print('Number of data points : ', data.shape[0])

Number of data points :  17593


Data points in our dataset is reduced to only 17K so save this dataset in pickels file folder such that we can use this file directly if any difficulties comes during the processing.

In [26]:
data.to_pickle('pickels/17k_apperal_data')

<pre>

In the previous stage, I sorted whole data in alphabetical order of  titles.Then, I removed titles which are  adjacent and very similar title(if prefixes of titles are same), I used above algorithm for removing similar products that's only because of time complextity of the code, since in next stage of the algorithm will take O(n^2) of time complextity to find similar products in entire dataset.

But there are some products whose titles are not adjacent but very similar.

Examples:

Titles-1
86261.  UltraClub Women's Classic Wrinkle-Free Long Sleeve Oxford Shirt, Pink, XX-Large
115042. UltraClub Ladies Classic Wrinkle-Free Long-Sleeve Oxford Light Blue XXL

TItles-2
75004.  EVALY Women's Cool University Of UTAH 3/4 Sleeve Raglan Tee
109225. EVALY Women's Unique University Of UTAH 3/4 Sleeve Raglan Tees
120832. EVALY Women's New University Of UTAH 3/4-Sleeve Raglan Tshirt

</pre>

So,I need to apply another algorithm/method to remove all those similar products which are not adjacent.

In [27]:
# This code snippet takes significant amount of time.
# O(n^2) time.
# Takes about an hour to run on a decent computer.

indices = []
for i,row in data.iterrows():
    indices.append(i)

stage2_dedupe_asins = []
while len(indices)!=0:
    i = indices.pop()
    stage2_dedupe_asins.append(data['asin'].loc[i])
    # consider the first apperal's title
    a = data['title'].loc[i].split()
    # store the list of words of ith string in a, ex: a = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']
    for j in indices:
        
        b = data['title'].loc[j].split()
        # store the list of words of jth string in b, ex: b = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']
        
        length = max(len(a),len(b))
        
        # count is used to store the number of words that are matched in both strings
        count  = 0

        # itertools.zip_longest(a,b): will map the corresponding words in both strings, it will appened None in case of unequal strings
        # example: a =['a', 'b', 'c', 'd']
        # b = ['a', 'b', 'd']
        # itertools.zip_longest(a,b): will give [('a','a'), ('b','b'), ('c','d'), ('d', None)]
        for k in itertools.zip_longest(a,b): 
            if (k[0]==k[1]):
                count += 1

        # if the number of words in which both strings differ are < 3 , I am considering it as those two apperals are same, hence we are ignoring them
        if (length - count) < 3:
            indices.remove(j)

Now reduce the dataset with only those products which have the same asins in the deduped dataset in previous steps

In [28]:
data = data.loc[data['asin'].isin(stage2_dedupe_asins)]
print('Number of data points after stage two of dedupe: ',data.shape[0])

Number of data points after stage two of dedupe:  16435


from 17k apperals it reduced to 16k apperals so save it in pickel file folder such that we can use it at any point of time directly this processsed dataset.

In [29]:
data.to_pickle('pickels/16k_apperal_data')